# loading ground truth data from csv file

`load_ground_truth_events` returns a list of `load_data.Event` wich contains metadata about a series of frames that may or may not show trophallaxis.
each `Event` has a property `observations`, which is a list of `load_data.Observation` which contains metadata about one frame.
The csv format is the one used by Andreas Berg, see help output for details.

In [2]:
from load_data import load_ground_truth_events
help(load_ground_truth_events)

Help on function load_ground_truth_events in module load_data:

load_ground_truth_events(csv_path:str, padding:int) -> [<class 'load_data.Event'>]
    Return a list of Event objects, one Event per row in the csv file.
    Args:
        csv_path: path to the ground truth data. required columns are: 
            track_id_combination, bee_a_detection_ids,bee_b_detection_ids, 
            human_decidable_interaction, trophallaxis_observed, 
            trophallaxis_start_frame_nr, trophallaxis_end_frame_nr
        padding: set to a positive value to add extra frames at the beginning and end of each event.



## saving the images
this will save the images for each event including padding frames before the first and after the last frame. images will be around 600 x 600 to leave room for augmentation.
this may take a very long time. for 2007 events it may take many days

In [3]:
from get_images import save_images
help(save_images)

events = [] # use return value of load_ground_truth_events
for i, event in enumerate(events):
    save_images(observations=event.observations, index=i, image_folder="raw_images")

Help on function save_images in module get_images:

save_images(observations:[<class 'load_data.Observation'>], index:int, image_folder='images_v2_y_events')
    Save an image for each Observation. The index is used for naming the folders.



# loading data from individual frames that are not grouped in Events

`load_candidates` calls `save_images` directly

In [4]:
from validation import load_candidates
help(load_candidates)

Help on function load_candidates in module validation:

load_candidates(frame_padding_length:int=20, json_file_path:str='/mnt/storage/david/cache/beesbook/trophallaxis/candidate_events.json', output_folder:str='candidates')
    load images based on json file. the images will need further preprocessing before use.
    json file format:
        [[frame_id, bee_id0, bee_id1, x1, y1, x2, y2, cam_id]]



# preprocessing
after this step, the images are ready for training. this step also takes a long time, but not more than a few hours.

In [5]:
from preprocess_images import preprocess_images
help(preprocess_images)

Help on function preprocess_images in module preprocess_images:

preprocess_images(image_folder:str, padding:int=16)
    crop and rotate all images and save the processed images 
    in two folders with the name {image_folder}_pad{padding}[_invert]
    Args:
        image_folder: input image path
        padding: padding that is left around the image on each side.
                 so e.g. 128x128 with padding 16 leads to 160x160



# training


In [6]:
from train import train
help(train)

Help on function train in module train:

train(seed, rca, item_depth, drop_frames_around_trophallaxis:bool, auto_archive=True, clahe=False, random_rotation_max=0, model_parameters=None, num_epochs=50, log_path='trainlog.txt', stats_path='train_stats.csv', batch_size=64, network=None, save_last_model=False)
    train a network, save stats, maybe save the model.
    Args:
        seed: determines which events go to test and which to train, for cross validaiton.
        rca: random crop amplitude, amount of random crops, set to 0 to disable.
        item_depth: how many images should the network see for each frame. 
                    if set to 3 the net sees the center frame and one frame before and after.
                    if item_depth is high, training takes longer.
        drop_frames_around_trophallaxis: if true ignore all negative frames of all positive events.
        auto_archive: if true the stats files get moved automatically to an archive folder
        clahe: apply clahe o

In [7]:
from train import cross_validate
help(cross_validate)

Help on function cross_validate in module train:

cross_validate(num_runs=10, **kwargs)
    run train num_runs times with different seeds for cross validation

